# Checking the directory and imported files

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing necessary libraries and packages

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers
from tokenizers import decoders, processors

# Importing train,test and validation datasets

In [ ]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2['toxic'] = train2.toxic.round().astype(int)

validation_data = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test_data = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [ ]:
train1.head()

In [ ]:
train2.head()

In [ ]:
validation_data.head()

In [ ]:
validation_data['lang'].value_counts()

# Let's use only the comments and toxic column from train1 and train2.

In [ ]:
train = pd.concat([train1[['comment_text', 'toxic']],train2[['comment_text', 'toxic']].query('toxic==1'),train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)])

In [ ]:
train.head()

In [ ]:
train.toxic.value_counts()

# Toggling the TPU Button literally does not turn ON them! Run the below lines to initialize the TPU (Thanks to Google!!)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def func1(texts, tokenizer,chunk_size=256,max_len=512):
    tokenizer.enable_truncation(max_length=max_len)
    tokenizer.enable_padding(max_length=max_len)
    all_ids = []
    
    for i in tqdm(range(0, len(texts),chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
def build_model_func(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    history = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=history)
    model.compile(Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
KDP = KaggleDatasets().get_gcs_path()

# The model we are going to use is XML_RoBERTa! Thanks to Facebook AI 

In [ ]:
MODEL = 'jplu/tf-xlm-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
def func2(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
x_train = func2(train['comment_text'].values, tokenizer, maxlen=192)
x_valid = func2(validation_data['comment_text'].values, tokenizer, maxlen=192)
x_test = func2(test_data['content'].values, tokenizer, maxlen=192)

y_train = train['toxic'].values
y_valid = validation_data['toxic'].values

In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
bs = BATCH_SIZE

In [ ]:
train_data = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(bs)
    .prefetch(AUTO)
)

valid_data = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(bs)
    .cache()
    .prefetch(AUTO)
)

test_data = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(bs)
)

In [ ]:
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model_func(transformer_layer, max_len=192)
model.summary()

In [ ]:
steps = x_train.shape[0] // bs
history = model.fit(
    train_data,
    steps_per_epoch=steps,
    validation_data=valid_data,
    epochs= 1
)

In [ ]:
steps = x_valid.shape[0] // bs
history_2 = model.fit(
    valid_data.repeat(),
    steps_per_epoch=steps,
    epochs=10
)

In [ ]:
sample_sub=pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')
sample_sub.head()

In [ ]:
sample_sub['toxic'] = model.predict(test_data, verbose=1)
sample_sub.to_csv('sample_submission.csv', index=False)

Sincere Thanks to Mr.Xhulu, whose notebooks have helped me infer knowledge of RoBERTa and its types!
https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras and 
https://www.kaggle.com/xhlulu/jigsaw-tpu-xlm-roberta

**Do upvote my notebook to encourage and support me in my early Kaggle days!**